In [5]:
import os
import csv
from collections import Counter
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#El csv debe estar en el mismo folder del archivo python
current_dir = os.getcwd()
filename = "Estadisticas.csv"

dataset_path = os.path.join(current_dir, filename)
df = pd.read_csv(dataset_path)

df.head()

,Delito,SubDelito,Fecha,Hora,Victima,SubVictima,Edad,Sexo,Nacionalidad,Provincia,Canton,Distrito
0,ASALTO,ARMA BLANCA,2025-01-01,06:00:00 - 08:59:59,PERSONA,PEATON [PERSONA],Mayor de edad,HOMBRE,NICARAGUA,SAN JOSE,SAN JOSE,NaN
1,ASALTO,ARMA BLANCA,2025-01-01,12:00:00 - 14:59:59,PERSONA,PEATON [PERSONA],Desconocido,HOMBRE,NICARAGUA,SAN JOSE,PEREZ ZELEDON,NaN
2,ASALTO,ARMA BLANCA,2025-01-01,03:00:00 - 05:59:59,PERSONA,PEATON [PERSONA],Mayor de edad,HOMBRE,COSTA RICA,SAN JOSE,DESAMPARADOS,NaN
3,ASALTO,ARMA BLANCA,2025-01-03,00:00:00 - 02:59:59,OTROS,BICICLETA/REPARTIDOR/MENSAJERO/COBRADOR [OTROS],Mayor de edad,HOMBRE,COSTA RICA,SAN JOSE,DESAMPARADOS,NaN
4,ASALTO,ARMA BLANCA,2025-01-01,03:00:00 - 05:59:59,VEHICULO,OTRO O INDETERMINADO [VEHICULO],Mayor de edad,HOMBRE,NICARAGUA,SAN JOSE,SAN JOSE,NaN


In [6]:
df.columns

Index(['Delito', 'SubDelito', 'Fecha', 'Hora', 'Victima', 'SubVictima', 'Edad',
       'Sexo', 'Nacionalidad', 'Provincia', 'Canton', 'Distrito'],
      dtype='object')